# Práctica 1. Aprendizaje Automático

Authors: Carlos Iborra Llopis (100451170), Alejandra Galán Arrospide (100451273) <br>
For additional notes and requirements: https://github.com/carlosiborra/Grupo02-Practica2-AprendizajeAutomatico

❗If you are willing to run the code yourself, please clone the full GitHub repository, as it contains the necessary folder structures to export images and results❗

# 0. Table of contents

- [Práctica 1. Aprendizaje Automático](#práctica-1-aprendizaje-automático)
  - [0. Table of contents](#0-table-of-contents)
  - [1. Requirements](#1-requirements)
  - [2. Reading the datasets](#2-reading-the-datasets)
  - [3. Exploratory Data Analysis](#3-EDA)
  - [4. Train-Test division ](#4-Train-Test-division )
  - [5. Basic Methods](#5-basic-methods)
  - [6. Reducing Dimensionality](#6-Reducing-Dimensionality)
  - [7. Advanced methods ](#7-Advanced-methods )
  - [8. Best model](#8-Best-model)
  - [9. Final Conclusions](#6-Final-Conclusions)

---
# 1. Requirements


In [4]:
""" Importing necessary libraries """
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
import scipy.stats as st
import scipy
import sklearn

from matplotlib.cbook import boxplot_stats as bps

### 1.1. Cleaning ../data/img/ folder
This way we avoid creating multiple images and sending the old ones to the trash.<br>
Also using this to upload cleaner commits to GitHub.


In [7]:
""" Cleaning the ../data/img/ folder """
import os
import glob

files = glob.glob("../data/img/*")
for f in files:
    if os.path.isfile(f) and f.endswith(".png"):
        os.remove(f)

---
# 2. Reading the datasets
Reading the datasets from the bz2 files, group 2.

In [6]:
import pickle

with open('../data/attrition_available_2.pkl', 'rb') as archivo_pkl:
    datos = pickle.load(archivo_pkl)

datos

In [ ]:
# Export the data to a csv file
# datos.to_csv('../data/attrition_available_2.csv', index=False)

---
# 3. EDA

<mark>(0,15 puntos) Hacer un EDA muy simplificado: cuántas instancias / cuantos 
atributos y de qué tipo (numéricos, ordinales, categóricos); columnas constantes o 
innecesarias; que proporción de missing values por atributo; tipo de problema: 
(clasificación o regresión); ¿es desbalanceado?</mark>

**Key Concepts of Exploratory Data Analysis**

- **2 types of Data Analysis**
  - Confirmatory Data Analysis
  - Exploratory Data Analysis
- **4 Objectives of EDA**
  - Discover Patterns
  - Spot Anomalies
  - Frame Hypothesis
  - Check Assumptions
- **2 methods for exploration**
  - Univariate Analysis
  - Bivariate Analysis
- **Stuff done during EDA**
  - Trends
  - Distribution
  - Mean
  - Median
  - Outlier
  - Spread measurement (SD)
  - Correlations
  - Hypothesis testing
  - Visual Exploration


## 3.0. Dataset preparation

To conduct exploratory data analysis (EDA) on our real data, we need to prepare the data first. Therefore, we have decided to separate the data into training and test sets at an early stage to avoid data leakage, which could result in an overly optimistic evaluation of the model, among other consequences. This separation will be done by dividing the data prematurely into training and test sets since potential data leakage can occur from the usage of the test partition, especially when including the result variable.

It is important to note that this step is necessary because all the information obtained in this section will be used to make decisions such as dimensionality reduction. Furthermore, this approach makes the evaluation more realistic and rigorous since the test set is not used until the end of the process.

<mark>Simplemente dividiremos los datos en un conjunto de train para entrenar y ajustar hiper-
Aprendizaje Automático
Práctica 2: Predicción de burnout
parámetros, y un conjunto de test en el que evaluaremos las distintas posibilidades 
que se probarán en la práctica. Hay que recordar que En problemas de clasificación 
desbalanceados hay que usar particiones estratificadas y métricas adecuadas 
(balanced_accuracy, f1, matriz de confusión). También es conveniente que los 
métodos de construcción de modelos traten el desbalanceo, usando por ejemplo 
el parámetro class_weight=”balanced”</mark>

## 3.1. Dataset description

In [15]:
num_filas = len(datos)


print("La variable datos tiene", num_filas - 1, "atributos.")

La variable datos tiene 4409 atributos.


In [17]:
num_atributos  = len(datos.keys())
# -1 Since it includes the result column
print("La variable datos tiene", num_atributos -1, "atributos.")

La variable datos tiene 30 atributos.


In [20]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4410 entries, 1 to 4409
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   hrs                      3639 non-null   float64
 1   absences                 3575 non-null   float64
 2   JobInvolvement           3585 non-null   float64
 3   PerformanceRating        3534 non-null   float64
 4   EnvironmentSatisfaction  3428 non-null   float64
 5   JobSatisfaction          3637 non-null   float64
 6   WorkLifeBalance          3620 non-null   float64
 7   Age                      3636 non-null   float64
 8   Attrition                4410 non-null   object 
 9   BusinessTravel           3687 non-null   object 
 10  Department               3575 non-null   object 
 11  DistanceFromHome         3681 non-null   float64
 12  Education                3628 non-null   float64
 13  EducationField           4410 non-null   object 
 14  EmployeeCount           

## 3.2. Missing values

Fist, we check the number the total number of missing values in the dataset in order to know if we have to clean the dataset or not.

In [23]:
datos.isna().sum()

hrs                        771
absences                   835
JobInvolvement             825
PerformanceRating          876
EnvironmentSatisfaction    982
JobSatisfaction            773
WorkLifeBalance            790
Age                        774
Attrition                    0
BusinessTravel             723
Department                 835
DistanceFromHome           729
Education                  782
EducationField               0
EmployeeCount              926
EmployeeID                   0
Gender                     887
JobLevel                   769
JobRole                    785
MaritalStatus              771
MonthlyIncome              831
NumCompaniesWorked         945
Over18                     859
PercentSalaryHike          753
StandardHours              919
StockOptionLevel           801
TotalWorkingYears          814
TrainingTimesLastYear      934
YearsAtCompany             732
YearsSinceLastPromotion    782
YearsWithCurrManager         0
dtype: int64

---
# 4. Train-Test division 

Since we are working with a time dependent data, we need to avoid mixing it. Also, we are required to add the first 10 years of data to the train set and the last 2 years to the test set. This means we are assigning a 83.333333 percent of the data to train and a 16.66666666 to test.

**Note**: This division was already done before the EDA. We overwrite it to start from a clean state.

<mark>En esta práctica la evaluación será más sencilla que en la primera. Simplemente 
dividiremos los datos en un conjunto de train para entrenar y ajustar hiper-
parámetros, y un conjunto de test en el que evaluaremos las distintas posibilidades 
que se probarán en la práctica. Hay que recordar que En problemas de clasificación 
desbalanceados hay que usar particiones estratificadas y métricas adecuadas 
(balanced_accuracy, f1, matriz de confusión). También es conveniente que los 
métodos de construcción de modelos traten el desbalanceo, usando por ejemplo 
el parámetro class_weight=”balanced”.</mark>

## 4.1. Train-Test split

## 4.2. Train-Test RMSE and MAE function

## 4.3. Print model results

## 4.4. Validation splits

## Decisions for all models 

For each possible method we have created two different models; One with predefined parameters and the second one with selected parameters. For each model we create a pipeline which includes the escaler ( except for trees and related ) and the model.  Note that we have selected RobustEscaler as our scaling method since we have found several outliers in the EDA. Secondly, we duplicate this two models per method and we add the selection of attributes. Note that the model with no selection of attributes and the one with selection of attributes have a double pipeline. Is a double pipeline since we use the output of the first pipeline ( best hiper-parameters ) directly into the second pipeline in order to avoid innecesary computing cost.

We have decided to train all models in the most similar way possible in order for the results to be comparable. This way, all models with selected parameters use RandomSearch in order to avoid unnecessary computational cost while still producing good results. Secondly, we have decided to use TimeSeriesSplit, which is a useful method when working with time-related data. We also perform a cross-validation within the parameter search in order to avoid optimistic scoring for some parameters. For all models, we are using a 5-fold cross-validation. We also decided to use NMAE as our method for testing error since it provides an easy-to-understand score and reduces the weight of outliers (as observed during the EDA process).

In addition note that in order to create the predefined models we are using gridsearch with just one option in the param-grid. This help us stay consistent in the way we create and compare models, since it provides a way of using cross-validation within the function. 

---
# 5. Model construction

<mark>(1.3 puntos) Construcción de modelos: para esta práctica usaremos
LogisticRegression como método base (sin ajustar hiper-parámetros) y Boosting
como método avanzado (ajustando hiper-parámetros), a elegir. Es importante 
realizar los preprocesos que los datos necesiten, usando preferentemente 
pipelines. Como método de boosting, se puede elegir uno de entre los métodos de 
boosting disponibles en scikit-learn. Si además se usa uno de entre las librerías 
externas xgboost, lightgbm o catboost, se pueden sacar +0.35 puntos adicionale</mark>

<mark>(0.8 puntos) Usando algún método de selección de atributos de tipo filter
(SelectKBest) de entre los disponibles en sklearn (f_classif, 
mutual_info_classif o chi2), comprobad si se pueden mejorar los resultados del 
apartado anterior y extraer conclusiones sobre qué atributos son más importantes, 
al menos de acuerdo a estos métodos</mark>

## 5.1. Logistic Regression
Logistic regression with no hyperparameter tuning. It will be used as a baseline for the rest of the models.

### 5.1.1. Logistic Regression - Predefined parameters

#### 5.1.1.1. Logistic Regression - Predefined parameters - No attribute selection

#### 5.1.1.2. Logistic Regression - Predefined parameters - Attribute selection

## 5.2. Boosting
With <mark>(?)</mark> and without hyperparameter tuning.

<mark>Como método de boosting, se puede elegir uno de entre los métodos de 
boosting disponibles en scikit-learn. Si además se usa uno de entre las librerías 
externas xgboost, lightgbm o catboost, se pueden sacar +0.35 puntos adicionales.</mark>

### 5.2.1. Boosting - Predefined parameters

#### 5.2.1.1. Boosting - Predefined parameters - No attribute selection

#### 5.2.1.2. Boosting - Predefined parameters - Attribute selection

### 5.2.2. Boosting - Selected parameters

#### 5.2.2.1. Boosting - Selected parameters - No attribute selection

#### 5.2.2.2. Boosting - Selected parameters - Attribute selection

---
# X. Output the Jupyter Notebook as an HTML file

In [27]:
import os

# Export the notebook to HTML
os.system("jupyter nbconvert --to html model.ipynb --output ../data/html/model.html")
print("Notebook exported to HTML")

Notebook exported to HTML


Traceback (most recent call last):
  File "/home/carlosiborra/anaconda3/envs/ml_practica_2/bin/jupyter-nbconvert", line 6, in <module>
    from nbconvert.nbconvertapp import main
  File "/home/carlosiborra/anaconda3/envs/ml_practica_2/lib/python3.10/site-packages/nbconvert/__init__.py", line 4, in <module>
    from .exporters import *
  File "/home/carlosiborra/anaconda3/envs/ml_practica_2/lib/python3.10/site-packages/nbconvert/exporters/__init__.py", line 3, in <module>
    from .html import HTMLExporter
  File "/home/carlosiborra/anaconda3/envs/ml_practica_2/lib/python3.10/site-packages/nbconvert/exporters/html.py", line 11, in <module>
    from jinja2 import contextfilter
ImportError: cannot import name 'contextfilter' from 'jinja2' (/home/carlosiborra/anaconda3/envs/ml_practica_2/lib/python3.10/site-packages/jinja2/__init__.py)
